In [1]:
###### Import Packages
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup as bs
from matplotlib import pyplot as plt
import seaborn as sns
from itertools import permutations 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
##### Assign Variales to be refrenced later 
league_id = 95857
swid = '{DED0A88A-F2E0-4B08-B4F8-BCC3DF9A5AB9}'
espn_s2 = 'AEARUl7xJzs4n3EM03LeY14eets7CrWBN2ogTCLyuy%2BxwHe9EudN6gmidZxoRFBT0O7Gbj1LOjlwmtGIsI8yE0Lc%2FXCk%2B5xN%2BslZW61iAFmXWFf2HfccgNeIpSzK8wbPMYRqYQATZpMTkd4XnWv6zwibOrMFDnYBJC8k1ZG3aTjLNBGFgxF9BUlVY9DUpXwg3dm%2BP6QbNwJ2eddel4235uci72%2BsQVAGI3qfR2P4loAz25YHO5qJ1bnzl2vcBV%2F1rpqa6jyEbc0L0n044RpdY6Z9mGYk6sX%2FsEhk2NlvSChvRg%3D%3D'


In [3]:
def Owner_DFS(Season):
    if Season == 2020:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + \
        str(Season)+ "/segments/0/leagues/"+str(league_id)
    else:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
        str(league_id) + "?seasonId=" + str(Season)



    r = requests.get(url,
                     cookies={
                      'swid':swid, 
                      'espn_s2':espn_s2   
                     })
    if Season == 2020:
        d = r.json()
    else:
        d = r.json()[0]


    Owner_ID = []
    Team_ID = []
    abbrev = []
    location = []
    nickname = []

    for team in d['teams']:
        Owner_ID.append(team['owners'])
        Team_ID.append(team['id'])
        abbrev.append(team['abbrev'])
        location.append(team['location'])
        nickname.append(team['nickname'])

    Teams_DF = pd.DataFrame(list(zip(Owner_ID, Team_ID, abbrev, location, nickname)), 
                            columns=['Owner_Id', 'Team_Id', 'abbrev', 'location', 'nickname'])
    return(Teams_DF)

#### Function to Count number of wins for a particular trial (Schedule)

In [4]:
####### Keep
#### First create dataframe to be referenced for each team
def Results(year):
    if year == 2020:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + \
        str(year)+ "/segments/0/leagues/"+str(league_id)
    else:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
        str(league_id) + "?seasonId=" + str(year)

    r = requests.get(url,params={"view": "mMatchup"},
                     cookies={
                      'swid':swid, 
                      'espn_s2':espn_s2   
                     })
    if year == 2020:
        d = r.json()
    else:
        d = r.json()[0]

    Games_DF = [[
            game['matchupPeriodId'],
            game['home']['teamId'], game['home']['totalPoints'],
            game['away']['teamId'], game['away']['totalPoints']
        ] for game in d['schedule'][:78]]
    Games_DF = pd.DataFrame(Games_DF, columns=['Week', 'Home_Team', 'Home_Score', 'Away_Team', 'Away_Score'])

    Home_DF = Games_DF[["Week", "Home_Team", "Home_Score"]]
    Away_DF = Games_DF[["Week", "Away_Team", "Away_Score"]]

    Home_DF = Home_DF.rename(columns = {"Home_Team":"Team", "Home_Score":"Points_Scored_For"})
    Away_DF = Away_DF.rename(columns = {"Away_Team":"Team", "Away_Score":"Points_Scored_For"})
    Result_DF = pd.concat([Home_DF, Away_DF])
    return(Result_DF)


In [5]:
##### Keep
#### Now Function to count number of wins for certain trial
Result_DF = Results(2020)
def winCount(Season, Team, Opp_List):
    Sched_List = [1,2,3,4,5,6,7,8,9,10,11,12,13]
    Team_DF = Result_DF[Result_DF['Team'] == Team].sort_values(by='Week')
    O_Teams_DF = pd.DataFrame({'Week':Sched_List, 'Team':Opp_List})
    O_Teams_Results = pd.merge(Result_DF, O_Teams_DF, on = ['Week', 'Team'], how = 'inner').sort_values(by='Week')
    Schedule_Results = pd.merge(Team_DF, O_Teams_Results, on = ['Week'], how = 'left').sort_values(by='Week')
    winTotal = len(Schedule_Results[Schedule_Results['Points_Scored_For_x'] > Schedule_Results['Points_Scored_For_y']].index)
    return(winTotal)
    

In [6]:
#### Keep
##### Get win count for a certain Team for a random schedule
def RandomWins(Season, Team):
    Team_List = [1,2,3,4,5,6,7,8,9,10,11,12]
    Team_List = list(filter(lambda x: x != Team, Team_List))
    np.random.shuffle(Team_List)
    Opp_List = Team_List
    Opp_List.extend(Opp_List[:2])
    win_Total = winCount(Season, Team, Opp_List)
    return(win_Total)

In [7]:
#### Keep
#### Loop through and return average and SD
def ExpectedWins(Season, Team, Trials):
    Trial_Count = 0
    Win_Count_List = []
    while Trial_Count <= Trials:
        winCount = RandomWins(Season, Team)
        Win_Count_List.append(winCount)
        Trial_Count = Trial_Count + 1
        #if Trial_Count % 50 == 0:
        #    print(Trial_Count)
    ExpectedWinValue = np.average(Win_Count_List)
    SD_WinValue = np.std(Win_Count_List)
    return(ExpectedWinValue, SD_WinValue)


In [8]:
# ##### Lets plot to see what the best number of trials for optimal accuracy speed trade off
# Trial_Number_Test_List = range(100,20000, 100)
# plot_List = []
# for trial in Trial_Number_Test_List:
#     EW, SD = ExpectedWins(2020, 1, trial)
#     plot_List.append(EW)

# plt.plot(Trial_Number_Test_List,plot_List)
# plt.title('Sample Accuracy')
# plt.xlabel('Number of Trials')
# plt.ylabel('E[W]')
# plt.show()

In [9]:
def ActualWinsCount(year, team):
    if year == 2020:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + \
        str(year)+ "/segments/0/leagues/"+str(league_id)
    else:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
        str(league_id) + "?seasonId=" + str(year)


    r = requests.get(url,params={"view": "mMatchup"},
                     cookies={
                      'swid':swid, 
                      'espn_s2':espn_s2   
                     })
    if year == 2020:
        d = r.json()
    else:
        d = r.json()[0]

    Games_DF = [[
            game['matchupPeriodId'],
            game['home']['teamId'], game['home']['totalPoints'],
            game['away']['teamId'], game['away']['totalPoints']
        ] for game in d['schedule'][:78]]
    Games_DF = pd.DataFrame(Games_DF, columns=['Week', 'Home_Team', 'Home_Score', 'Away_Team', 'Away_Score'])

    Home_DF = Games_DF[["Week", "Home_Team", "Home_Score", "Away_Team", "Away_Score"]]
    Away_DF = Games_DF[["Week", "Away_Team", "Away_Score", "Home_Team", "Home_Score"]]

    Home_DF = Home_DF.rename(columns = {"Home_Team":"Team", "Home_Score":"Points_Scored_For", "Away_Team": "Opposing_Team", "Away_Score":"Points_Scored_Against"})
    Away_DF = Away_DF.rename(columns = {"Away_Team":"Team", "Away_Score":"Points_Scored_For", "Home_Team": "Opposing_Team", "Home_Score":"Points_Scored_Against"})
    Result_DF = pd.concat([Home_DF, Away_DF])
    Win_Count = len(Result_DF[(Result_DF["Points_Scored_For"] > Result_DF["Points_Scored_Against"]) & (Result_DF["Team"] == team)])
    return(Win_Count)

In [10]:
def ZScore_Win(Observed, Expected, SD):
    ZScore = (Observed - Expected)/SD
    return(ZScore)

In [11]:
def LuckRankings(Season):
    teams = [1,2,3,4,5,6,7,8,9,10,11,12]
    ZScoreList = []
    WinDiffs = []
    for team in teams:
        EW, SD_EW = ExpectedWins(Season, team, 100)
        AW = ActualWinsCount(Season, team)
        ZScore = ZScore_Win(AW, EW, SD_EW)
        ZScoreList.append(ZScore)
        WinDiff = AW - EW
        WinDiffs.append(WinDiff)
    Luck_DF = pd.DataFrame({"Team": teams, "Luck":ZScoreList, "Wins Over Expected":WinDiffs, "Season":Season})
    Luck_DF = pd.merge(Luck_DF, Owner_DFS(Season)[['Team_Id','abbrev']], left_on = 'Team', right_on = 'Team_Id' ,how = 'left')
    return(Luck_DF.sort_values(by="Luck"))    

In [12]:
def LuckRankings_All():
    Seasons = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
    Lucky_DF = LuckRankings(Seasons[0])
    for Season in Seasons[1:]:
        Season_Lucky_DF = LuckRankings(Season)
        Lucky_DF = pd.concat([Lucky_DF, Season_Lucky_DF])
    return(Lucky_DF.sort_values(by="Luck"))    

In [13]:
LuckRankings(2020)

,Team,Luck,Wins Over Expected,Season,Team_Id,abbrev
2,3,-1.469144,-2.237624,2020,3,Rick
6,7,-1.028588,-1.336634,2020,7,TUCK
5,6,-0.908643,-1.336634,2020,6,NEB
4,5,-0.267485,-0.366337,2020,5,SCNO
9,10,-0.093065,-0.138614,2020,10,CB
0,1,0.015637,0.019802,2020,1,Wood
11,12,0.190823,0.287129,2020,12,Meer
1,2,0.217310,0.306931,2020,2,Wutt
3,4,0.267566,0.376238,2020,4,CHIN
8,9,0.597550,0.673267,2020,9,BINK
